In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [103]:
class GeneticAlgorithm:
    def __init__(self, pop_size, x_range, y_range, max_iter=10000, cross_rate=0.7, mutation_rate=0.001):
        self.pop_size = pop_size
        self.x_range = x_range
        self.y_range = y_range
        self.max_iter = max_iter
        self.cross_rate = cross_rate
        self.mutation_rate = mutation_rate
    
    def _initialize_population(self):
        population = np.random.uniform(size=(self.pop_size, 2))
        
        x_interval = self.x_range[1] - self.x_range[0]
        y_interval = self.y_range[1] - self.y_range[0]
        
        # Map [0, 1] values to corresponding ranges
        population[:, 0] = population[:, 0] * x_interval + self.x_range[0]
        population[:, 1] = population[:, 1] * y_interval + self.y_range[0]
        
        return population
    
    def _evaluate_population(self, population):
        x_values = population[:, 0]
        y_values = population[:, 1]
        
        return 21.5 + x_values * np.sin(4 * np.pi * x_values) + y_values * np.sin(20 * np.pi * y_values)
    
    def _sort_population_fitness(self, population, fitness):
        sort_idx = np.argsort(fitness)[::-1]
        
        return population[sort_idx], fitness[sort_idx]
    
    def _selection_schema(self, population, fitness):
        # The selection schema is based on the binary torunament
        new_population = []
        
        for _ in range(self.pop_size):
            idx_1, idx_2 = np.random.choice(self.pop_size, 2)
            new_population.append(population[idx_1] if fitness[idx_1] > fitness[idx_2] else population[idx_2])
        
        new_population = np.array(new_population)
        
        return new_population
    
    def _blx_alpha_crossover(self, parents, alpha=0.2):
        # Compute min and max values column-wise
        c_min = np.min(parents, axis=0)
        c_max = np.max(parents, axis=0)

        # Compute interval
        i = c_max - c_min
        
        child_1 = np.random.uniform(c_min - i * alpha, c_max + i * alpha)
        child_2 = np.random.uniform(c_min - i * alpha, c_max + i * alpha)
        
        children = np.vstack((child_1, child_2))
        children = self._clip_values(children)
        
        return children
        
    
    def _cross_schema(self, population):
        prob_cross = np.random.uniform(size=self.pop_size)
        cross_idx = np.where(prob_cross < self.cross_rate)[0]
        
        # If length is even, remove last element
        if len(cross_idx) % 2 != 0:
            cross_idx = cross_idx[:-1]
        
        cross_idx = np.random.permutation(cross_idx)
        cross_idx = cross_idx.reshape(-1, 2)
        
        for couple_idx in cross_idx:
            parents = population[couple_idx]
            children = self._blx_alpha_crossover(parents)
            
            population[couple_idx] = children
            
        return population
    
    
    def _mutation(self, population, mean=0., sigma=0.7):
        mutation_prob = np.random.uniform(size=population.shape)
        mutation_idx = np.where(mutation_prob < self.mutation_rate)
        
        population[mutation_idx] += np.random.normal(mean, sigma, size=mutation_idx[0].shape)
        population = self._clip_values(population)
        
        return population
    
    
    def _clip_values(self, population):
        population[:, 0] = np.clip(population[:, 0], *self.x_range)
        population[:, 1] = np.clip(population[:, 1], *self.y_range)
        
        return population
    
    def _elitism(self, old_population, old_fitness, new_population, new_fitness):
        if old_fitness[0] > new_fitness[0]:
            new_population[-1] = old_population[0]
            new_fitness[-1] = old_fitness[0]
            
            new_population, new_fitness = self._sort_population_fitness(new_population, new_fitness)
        
        return new_population, new_fitness
    
        
    def train_predict(self):
        population = self._initialize_population()
        fitness = self._evaluate_population(population)
        
        # Sort population and fitness by fitness value
        population, fitness = self._sort_population_fitness(population, fitness)
        
        for i in range(self.max_iter):
            # 1. Selection
            new_population = self._selection_schema(population, fitness)
            new_fitness = self._evaluate_population(new_population)
            new_population, new_fitness = self._sort_population_fitness(new_population, new_fitness)
            
            # 2. Cross
            new_population = self._cross_schema(new_population)
            
            # 3. Mutate
            new_population = self._mutation(new_population)
            
            # 4. Evaluation and sorting
            new_fitness = self._evaluate_population(new_population)
            new_population, new_fitness = self._sort_population_fitness(new_population, new_fitness)
            population, fitness = self._elitism(population, fitness, new_population, new_fitness)
            
            if i % 1000 == 0:
                print('Current best fitness: ', fitness[0], population[0])
        
        return population[0]

In [104]:
ga = GeneticAlgorithm(50, (-3., 12.11), (4.5, 5.8))

In [105]:
ga.train_predict()

Current best fitness:  36.523458741361786 [11.11229939  4.83417812]
Current best fitness:  38.82052162464592 [12.11        5.42504116]
Current best fitness:  38.82052162464592 [12.11        5.42504116]
Current best fitness:  38.82052162464592 [12.11        5.42504116]
Current best fitness:  39.12052072869683 [12.11        5.72504424]
Current best fitness:  39.12052072869683 [12.11        5.72504424]
Current best fitness:  39.12052072869683 [12.11        5.72504424]
Current best fitness:  39.12052072869683 [12.11        5.72504424]
Current best fitness:  39.12052072869683 [12.11        5.72504424]
Current best fitness:  39.12052072869683 [12.11        5.72504424]


array([12.11      ,  5.72504424])